In [1]:
!pip install google-play-scraper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00


In [4]:
from google_play_scraper import reviews, Sort
import pandas as pd
import os

def scrape_reviews(app_id, bank_name, num_reviews=500):
    all_reviews = []
    token = None

    while len(all_reviews) < num_reviews:
        result, token = reviews(
            app_id,
            lang='en',
            country='us',
            sort=Sort.NEWEST,
            count=200,  # max per call
            continuation_token=token
        )

        for review in result:
            all_reviews.append({
                "review": review['content'],
                "rating": review['score'],
                "date": review['at'].date().isoformat(),
                "bank": bank_name,
                "source": "Google Play"
            })
            if len(all_reviews) >= num_reviews:
                break

    return pd.DataFrame(all_reviews)


In [6]:
df_cbe = scrape_reviews("com.combanketh.mobilebanking", "CBE", num_reviews=500)
df_boa = scrape_reviews("com.boa.boaMobileBanking", "BOA", num_reviews=500)
df_dashen = scrape_reviews("com.cr2.amolelight", "Dashen", num_reviews=400)


In [7]:
df_all = pd.concat([df_cbe, df_boa, df_dashen], ignore_index=True)
df_all.drop_duplicates(inplace=True)
df_all.dropna(subset=["review", "rating", "date"], inplace=True)


In [8]:
df_all['date'] = pd.to_datetime(df_all['date']).dt.strftime('%Y-%m-%d')


In [9]:
df_all.to_csv("cleaned_reviews.csv", index=False)
